<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |████████████████████████████████| 3.0 MB 59.3 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd

In [3]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [38]:
import re
from datetime import datetime

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/"
ticker_path = path + "Tickers_reduced_full.csv"

tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()

In [39]:
for ticker in tickers_list:
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  print(file_path)

https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AAL_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AAPL_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/ACA_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/ADS_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AIR_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AL_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AMC_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AMD_tweets_2020_cleaned.csv
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AMH_tweets_2020_cleaned.csv
https://raw.githubuserconten

In [ ]:
texts = df["Text_clean"]

chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

In [16]:
print(file_path)

results_list = []

for idx in range(len(chunks)):

  cp1 = datetime.now()
  
  chunk = chunks[idx].tolist()

  candidate_labels = ["positive", "negative", "neutral"]

  hypothesis_template = "The sentiment of this tweet is {}."

  #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
  #result = dicts["labels"][0]
  #results_list.append(result)
  
  list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)
  new_list = [i["labels"][0] for i in list_results_dicts]
  results_list.extend(new_list)

  cp2 = datetime.now()

  print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/AAL_tweets_2020_cleaned.csv
1/791 --- 0:00:03.095393
2/791 --- 0:00:03.266033
3/791 --- 0:00:03.274121
4/791 --- 0:00:03.195990
5/791 --- 0:00:02.957517
6/791 --- 0:00:03.397165
7/791 --- 0:00:03.023309
8/791 --- 0:00:03.402825
9/791 --- 0:00:03.258405
10/791 --- 0:00:02.949549
11/791 --- 0:00:03.099121
12/791 --- 0:00:02.967838
13/791 --- 0:00:03.258671
14/791 --- 0:00:03.255261
15/791 --- 0:00:03.042592
16/791 --- 0:00:03.074463
17/791 --- 0:00:03.124302
18/791 --- 0:00:02.944133
19/791 --- 0:00:02.879365
20/791 --- 0:00:02.792243
21/791 --- 0:00:03.127182
22/791 --- 0:00:03.123088
23/791 --- 0:00:02.910730
24/791 --- 0:00:03.187980
25/791 --- 0:00:02.958215
26/791 --- 0:00:02.960336
27/791 --- 0:00:02.861609
28/791 --- 0:00:03.121952
29/791 --- 0:00:03.115863
30/791 --- 0:00:02.961445
31/791 --- 0:00:02.928582
32/791 --- 0:00:02.928822
33/791 --- 0:00:03.118031
34/791 --- 0:00:02.764021
35/791 --- 0:00:0

In [17]:
df["Sentiment"] = results_list

newfilename = ticker + str("_tweets_2020_tagged.csv")

df.to_csv(newfilename, index = False)